In [ ]:
%matplotlib inline
import os
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import itertools
import matplotlib.ticker as mtick  # Import the formatter for percentages
import matplotlib as mpl

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.ticker import FixedLocator
from matplotlib.ticker import StrMethodFormatter
# Set the font family to serif
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
%matplotlib inline
import os
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import itertools
import matplotlib.ticker as mtick  # Import the formatter for percentages
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.ticker import FixedLocator
from matplotlib.ticker import StrMethodFormatter

# Set the font family to 'Times New Roman' for better readability
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
# Set the file path for the dataset
file_path = r'C:\Users\labo\OneDrive - Nexus365\dissertation\LB_events_all_cleaned.csv'

# Load the dataset with appropriate data types and date parsing
dtype_dict = {"receive_receipt_token": "object"}
parse_dates = ["datetime"]
lb = pd.read_csv(file_path, dtype=dtype_dict, parse_dates=parse_dates)

In [ ]:
# Truncate the dataset to match the same period as used for correlation analysis
lb = lb[lb['datetime'] <= '2023-08-19 23:59:59']

In [ ]:
# Create a DataFrame containing only 'liquidation' records
liquidations_df = lb[lb['type'] == 'liquidation']

In [ ]:
# Drop columns containing NaN values from the 'liquidations_df'
liquidations_df = liquidations_df.dropna(axis=1)

In [ ]:
# Identify all unique addresses from the 'liquidation_caller_address' column
unique_liquidators = liquidations_df['liquidation_caller_address'].unique()

In [ ]:
# Create a mapping from addresses to integer values starting from 1
# (early liquidators have lower numbers)
address_mapping = {address: i + 1 for i, address in enumerate(unique_liquidators)}

In [ ]:
# Substitute the addresses in the 'liquidation_caller_address' column with their mapped values
liquidations_df['liquidation_caller_address'] = liquidations_df['liquidation_caller_address'].map(address_mapping)

# Create the new column 'block_number_ordering', which is derived from 'block_number' and 'log_index' divided by 10000
liquidations_df['block_number_ordering'] = liquidations_df['block_number'] + liquidations_df['log_index'] / 10000

# Sort the DataFrame by 'asset_symbol' and 'block_number_ordering'
liquidations_df_sorted = liquidations_df.sort_values(by=['asset_symbol', 'block_number_ordering'])

# Compute the difference between consecutive 'block_number_ordering' values within each 'asset_symbol' group
liquidations_df_sorted['block_diff'] = liquidations_df_sorted.groupby('asset_symbol')['block_number_ordering'].diff()

# Identify rows where the difference ('block_diff') is less than or equal to 25 blocks (equivalent to 5 minutes)
liquidations_df_sorted['part_of_wave_tail'] = liquidations_df_sorted['block_diff'] <= 25

In [ ]:
# Add a 'part_of_wave_head' column, which is a shifted version of 'part_of_wave_tail'
liquidations_df_sorted['part_of_wave_head'] = liquidations_df_sorted['part_of_wave_tail'].shift(-1)

# Identify rows that are part of a wave (either part of the wave tail or wave head)
liquidations_df_sorted['part_of_wave'] = liquidations_df_sorted['part_of_wave_tail'] | liquidations_df_sorted['part_of_wave_head']

# Drop the 'part_of_wave_head' column
liquidations_df_sorted.drop(columns=['part_of_wave_head'], inplace=True)

# Add columns to help identify the start of a wave
liquidations_df_sorted['next_part_of_wave_tail'] = liquidations_df_sorted['part_of_wave_tail'].shift(-1)
liquidations_df_sorted['next_asset_symbol'] = liquidations_df_sorted['asset_symbol'].shift(-1)

# Identify the start of a wave based on the given conditions
liquidations_df_sorted['wave_start'] = (~liquidations_df_sorted['part_of_wave_tail']) & (liquidations_df_sorted['next_part_of_wave_tail']) & (liquidations_df_sorted['asset_symbol'] == liquidations_df_sorted['next_asset_symbol'])

# Drop the temporary columns used for the calculation
liquidations_df_sorted.drop(columns=['next_part_of_wave_tail', 'next_asset_symbol'], inplace=True)

# Create a 'wave_ID' column by cumulatively summing the 'wave_start' column
liquidations_df_sorted['wave_ID'] = liquidations_df_sorted['wave_start'].cumsum()

# Set 'wave_ID' to NaN for rows that are not part of a wave
liquidations_df_sorted.loc[~liquidations_df_sorted['part_of_wave'], 'wave_ID'] = np.nan

total_liquidations = liquidations_df_sorted.shape[0]
total_wave_liquidations = liquidations_df_sorted['part_of_wave'].sum()
total_wave_liquidations_relative = total_wave_liquidations / total_liquidations
total_waves = liquidations_df_sorted['wave_ID'].nunique()

# Find the maximum number of liquidations in a wave
# Group by 'wave_ID' and count the number of liquidations in each wave
liquidations_per_wave = liquidations_df_sorted.groupby('wave_ID').size()
max_liquidations_in_wave = liquidations_per_wave.max()

# Print the results
print(f"Total number of liquidations: {total_liquidations}")
print(f"Total number of waves: {total_waves}")
print(f"Total number of liquidations that are part of a wave: {total_wave_liquidations} (or {total_wave_liquidations_relative:.2%} of total liquidations)")
print(f"Number of liquidations in the wave with the most liquidations: {max_liquidations_in_wave}")

In [ ]:
# Filter the DataFrame for rows that are part of a wave
wave_liquidations = liquidations_df_sorted[liquidations_df_sorted['part_of_wave']]

# Extract the relevant columns ('datetime', 'block_number', 'asset_symbol', 'wave_ID', 'liquidation_caller_address') and reset the index
wave_liquidations_long = wave_liquidations[['datetime', 'block_number', 'asset_symbol', 'wave_ID', 'liquidation_caller_address']].reset_index()

# Add a new column 'position_in_wave', which represents the position of the address in that wave
wave_liquidations_long['position_in_wave'] = wave_liquidations_long.groupby('wave_ID').cumcount() + 1

# Rearrange the columns for better readability
wave_liquidations_long = wave_liquidations_long[['wave_ID', 'liquidation_caller_address', 'position_in_wave', 'datetime', 'block_number', 'asset_symbol']]

In [ ]:
### 1. Approach: Identifying connections

# Group by 'liquidation_caller_address' and count the number of liquidations for each liquidator
liquidator_stats = liquidations_df.groupby('liquidation_caller_address').size().reset_index(name='total_liquidations')

# Count the number of times each address was part of a wave
wave_counts = wave_liquidations_long['liquidation_caller_address'].value_counts().reset_index()
wave_counts.columns = ['liquidation_caller_address', 'total_wave_liquidations']

# Merge this count with the original DataFrame 'liquidator_stats'
liquidator_stats = pd.merge(liquidator_stats, wave_counts, on='liquidation_caller_address', how='left')

# Fill NaN values with 0 for addresses that were never part of a wave
liquidator_stats['total_wave_liquidations'].fillna(0, inplace=True)
liquidator_stats['total_wave_liquidations'] = liquidator_stats['total_wave_liquidations'].astype(int)

# Re-filter addresses for the most active 20% liquidators based on specific criteria
selected_addresses = liquidator_stats[(liquidator_stats['total_liquidations'] >= 22) & (liquidator_stats['total_wave_liquidations'] >= 9)]['liquidation_caller_address'].tolist()
filtered_waves = wave_liquidations_long[wave_liquidations_long['liquidation_caller_address'].isin(selected_addresses)] # Filter only the rows of the selected callers

# Constructing the graph with 2^-distance weight decay within 5 blocks distance
G = nx.DiGraph()

for address in selected_addresses:
    address_waves = filtered_waves[filtered_waves['liquidation_caller_address'] == address]
    
    for _, row in address_waves.iterrows():
        wave_id = row['wave_ID']
        block_num = row['block_number']
        
        subsequent_rows = filtered_waves[(filtered_waves['wave_ID'] == wave_id) & 
                                         (filtered_waves['block_number'] > block_num) & 
                                         (filtered_waves['block_number'] <= block_num + 5)]
        
        for _, subsequent_row in subsequent_rows.iterrows():
            subsequent_address = subsequent_row['liquidation_caller_address']
            distance = subsequent_row['block_number'] - block_num
            weight = 2**(-distance)
            
            if G.has_edge(address, subsequent_address):
                G[address][subsequent_address]['weight'] += weight
            else:
                G.add_edge(address, subsequent_address, weight=weight)

In [ ]:
# Calculate in-degrees and out-degrees for each node
in_degrees = dict(G.in_degree(weight='weight'))
out_degrees = dict(G.out_degree(weight='weight'))

# Create a DataFrame for better visualization
degree_df = pd.DataFrame({
    'In_Degree': in_degrees,
    'Out_Degree': out_degrees,
    'Net_Degree': {node: out_degrees[node] - in_degrees[node] for node in G.nodes()},
    'Abs_Degree': {node: out_degrees[node] + in_degrees[node] for node in G.nodes()}
}).sort_values(by='Abs_Degree', ascending=False)

# Generate the adjacency matrix for the graph
adjacency_matrix = nx.adjacency_matrix(G)

# Convert the adjacency matrix to a DataFrame for better readability
adjacency_df = pd.DataFrame(adjacency_matrix.todense(), index=G.nodes(), columns=G.nodes())

# Create adjacency_normalised_out_df by dividing rows by 'Out_Degree' values
adjacency_normalised_out_df = adjacency_df.divide(degree_df['Out_Degree'].reindex(adjacency_df.index).values, axis=0)

adjacency_normalised_out_df.fillna(0, inplace=True)

# Convert columns of adjacency_df to integers for matching with degree_df
adjacency_df.columns = adjacency_df.columns.astype(int)

# Perform vectorized normalization
adjacency_normalised_in_df = adjacency_df.divide(degree_df['In_Degree'].reindex(adjacency_df.columns).values, axis=1)

adjacency_normalised_in_df.fillna(0, inplace=True)

# Identify indices with 'Abs_Degree' greater than the median
median_abs_degree = degree_df['Abs_Degree'].median()
high_abs_degree_indices = degree_df[degree_df['Abs_Degree'] > median_abs_degree].index

# Filter adjacency_normalised_in_df and adjacency_normalised_out_df based on these indices
filtered_adjacency_normalised_in_df = adjacency_normalised_in_df.loc[high_abs_degree_indices, high_abs_degree_indices]
filtered_adjacency_normalised_out_df = adjacency_normalised_out_df.loc[high_abs_degree_indices, high_abs_degree_indices]

# Replace diagonal entries with 0s for both dataframes
filtered_adjacency_normalised_in_df.values[np.diag_indices_from(filtered_adjacency_normalised_in_df)] = 0
filtered_adjacency_normalised_out_df.values[np.diag_indices_from(filtered_adjacency_normalised_out_df)] = 0

In [ ]:
### Plotting the graph G_in

# Create a directed graph based on filtered_adjacency_normalised_in_df
G_in = nx.DiGraph()

# Add edges where cell value is at least 1/3
for i, row in filtered_adjacency_normalised_in_df.iterrows():
    for j, value in row.iteritems():
        if value >= 0.3:
            G_in.add_edge(i, j, weight=value)

# Set the font family to 'serif' for better readability
plt.rcParams['font.family'] = 'serif'

# Create a color map based on 'total_wave_liquidations' values
color_map_in = []
for node in G_in.nodes:
    liquidations = liquidator_stats.loc[liquidator_stats['liquidation_caller_address'] == node, 'total_wave_liquidations']
    if len(liquidations) > 0:
        liquidations = liquidations.iloc[0]
        if liquidations < 250:
            color_map_in.append('lightgrey')
        elif liquidations < 500:
            color_map_in.append('skyblue')
        else:
            color_map_in.append((0.27, 0.51, 0.71, 0.9))  # Steelblue with alpha 90%
    else:
        color_map_in.append('gray')  # Default color for nodes not in liquidator_stats

# Plot the graph
plt.figure(figsize=(10, 10))  # Adjust the figure size for better visualization
pos_in = nx.circular_layout(G_in)

# Separate edges into two groups based on weight
dashed_edges_in = [(u, v) for u, v, value in G_in.edges.data('weight') if value < 0.5]
solid_edges_in = [(u, v) for u, v, value in G_in.edges.data('weight') if value >= 0.5]

# Draw solid edges
nx.draw_networkx_edges(G_in, pos_in, edgelist=solid_edges_in, width=2.0, alpha=0.5, edge_color='black', style='solid', arrowstyle='-|>', arrowsize=10, min_target_margin=11, min_source_margin=12)

# Draw dashed edges
nx.draw_networkx_edges(G_in, pos_in, edgelist=dashed_edges_in, width=2.0, alpha=0.5, edge_color='black', style='dashed', arrowstyle='-|>', arrowsize=10, min_target_margin=11, min_source_margin=12)

# Draw nodes with color based on color_map
nx.draw_networkx_nodes(G_in, pos_in, node_size=500, node_color=color_map_in)
nx.draw_networkx_labels(G_in, pos_in, font_size=10, font_family='serif')

# Create custom legend items
legend_items_in = [Line2D([0], [0], color='black', alpha=0.5, linestyle='dashed', label='$\mathregular{Weight \in [0.3,0.5]}$'),
                Line2D([0], [0], color='black', alpha=0.5, linestyle='-', label='$\mathregular{Weight \in (0.5,1]}$')]

# Add color-coded nodes information to the legend
legend_items_in += [Line2D([0], [0], marker='o', color='lightgrey', label='Wave liquidations < 250', markersize=10, linestyle='None'),
                 Line2D([0], [0], marker='o', color='skyblue', label='$\mathregular{Wave\ liquidations \in [250,500)}$', markersize=10, linestyle='None'),
                 Line2D([0], [0], marker='o', color='steelblue', label='$\mathregular{Wave\ liquidations \geq 500}$', markersize=10, linestyle='None')]

plt.legend(handles=legend_items_in, loc='center', fontsize='x-large')  # Add the legend

# Turn off the axis frame
plt.axis('off')

# Save the plot to the specified directory
# plt.savefig(r'C:\Users\labo\OneDrive - Nexus365\Diss\exploratory analsis\plots\G_in.png', bbox_inches='tight', pad_inches=0.01)

# plt.title("Directed Graph from filtered_adjacency_normalised_in_df")
plt.show()

In [ ]:
### Plotting the graph G_out

# Create a directed graph based on filtered_adjacency_normalised_out_df
G_out = nx.DiGraph()

# Add edges where cell value is at least 1/3
for i, row in filtered_adjacency_normalised_out_df.iterrows():
    for j, value in row.iteritems():
        if value > 0.3:
            G_out.add_edge(i, j, weight=value)

# Set the font family to 'serif' for better readability
plt.rcParams['font.family'] = 'serif'

# Create a color map based on 'total_wave_liquidations' values
color_map_out = []
for node in G_out.nodes:
    liquidations = liquidator_stats.loc[liquidator_stats['liquidation_caller_address'] == node, 'total_wave_liquidations']
    if len(liquidations) > 0:
        liquidations = liquidations.iloc[0]
        if liquidations < 250:
            color_map_out.append('lightgrey')
        elif liquidations < 500:
            color_map_out.append('skyblue')
        else:
            color_map_out.append((0.27, 0.51, 0.71, 0.9))  # Steelblue with alpha 90%
    else:
        color_map_out.append('gray')  # Default color for nodes not in liquidator_stats

# Plot the graph
plt.figure(figsize=(10, 10))  # Adjust the figure size for better visualization
pos_out = nx.circular_layout(G_out)

# Separate edges into two groups based on weight
dashed_edges_out = [(u, v) for u, v, value in G_out.edges.data('weight') if value < 0.5]
solid_edges_out = [(u, v) for u, v, value in G_out.edges.data('weight') if value >= 0.5]

# Draw solid edges
nx.draw_networkx_edges(G_out, pos_out, edgelist=solid_edges_out, width=2.0, alpha=0.5, edge_color='black', style='solid', arrowstyle='-|>', arrowsize=10, min_target_margin=11, min_source_margin=12)

# Draw dashed edges
nx.draw_networkx_edges(G_out, pos_out, edgelist=dashed_edges_out, width=2.0, alpha=0.5, edge_color='black', style='dashed', arrowstyle='-|>', arrowsize=10, min_target_margin=11, min_source_margin=12)

# Draw nodes with color based on color_map
nx.draw_networkx_nodes(G_out, pos_out, node_size=500, node_color=color_map_out)
nx.draw_networkx_labels(G_out, pos_out, font_size=10, font_family='serif')

# Create custom legend items
legend_items_out = [Line2D([0], [0], color='black', linestyle='dashed', label='$\mathregular{Weight \in [0.3,0.5]}$'),
                Line2D([0], [0], color='black', linestyle='-', label='$\mathregular{Weight \in (0.5,1]}$')]

# Add color-coded nodes information to the legend
legend_items_out += [Line2D([0], [0], marker='o', color='lightgrey', label='Wave liquidations < 250', markersize=10, linestyle='None'),
                 Line2D([0], [0], marker='o', color='skyblue', label='$\mathregular{Wave\ liquidations \in [250,500)}$', markersize=10, linestyle='None'),
                 Line2D([0], [0], marker='o', color='steelblue', label='$\mathregular{Wave\ liquidations \geq 500}$', markersize=10, linestyle='None')]

plt.legend(handles=legend_items_out, loc='center', fontsize='x-large')  # Add the legend

# Turn off the axis frame
plt.axis('off')

# Save the plot to the specified directory
# plt.savefig(r'C:\Users\labo\OneDrive - Nexus365\Diss\exploratory analsis\plots\G_out.png', bbox_inches='tight', pad_inches=0.01)

# plt.title("Directed Graph from filtered_adjacency_normalised_out_df")
plt.show()


In [ ]:
# Creating the combined graph G_inout as an undirected graph
G_inout = nx.Graph()

# Create a set to hold the undirected edges that should be added to G_inout
edges_inout = set()

# Iterate through the directed edges in G_in
for u, v in G_in.edges():
    # Check if the opposite-directed edge exists in G_out
    if G_out.has_edge(v, u):
        edges_inout.add((u, v))

# Iterate through the directed edges in G_out
for u, v in G_out.edges():
    # Check if the opposite-directed edge exists in G_in
    if G_in.has_edge(v, u):
        edges_inout.add((u, v))

# Add the edges to G_inout
G_inout.add_edges_from(edges_inout)

# Set the font family to 'serif' for better readability
plt.rcParams['font.family'] = 'serif'

# Create the color map based on the total_wave_liquidations values
color_map_inout = []
for node in G_inout.nodes():
    liquidations = liquidator_stats.loc[liquidator_stats['liquidation_caller_address'] == node, 'total_wave_liquidations']
    if len(liquidations) > 0:
        liquidations = liquidations.iloc[0]
        if liquidations < 250:
            color_map_inout.append('lightgrey')
        elif liquidations < 500:
            color_map_inout.append('skyblue')
        else:
            color_map_inout.append('steelblue')
    else:
        color_map_inout.append('gray')  # Default color for nodes not in liquidator_stats

# Plotting the graph
plt.figure(figsize=(27, 9))  # Adjust the figure size for better visualization
pos_inout = nx.spring_layout(G_inout, k=0.65, seed=168)  # Layout for the nodes

# Draw the graph
nx.draw(G_inout, pos_inout, node_color=color_map_inout, edge_color=((0, 0, 0, 0.5)), width=4, with_labels=True, node_size=5000, font_size=30, font_family='serif')

# Add the legend
legend_items_inout = [Line2D([0], [0], marker='o', color='lightgrey', label='Wave liquidations < 250', markersize=30, linestyle='None'),
                     Line2D([0], [0], marker='o', color='skyblue', label='$\mathregular{Wave\ liquidations \in [250,500)}$', markersize=30, linestyle='None'),
                     Line2D([0], [0], marker='o', color='steelblue', label='$\mathregular{Wave\ liquidations \geq 500}$', markersize=30, linestyle='None')]

plt.legend(handles=legend_items_inout, loc='best', frameon=True, facecolor='none')
#plt.title("Combined Undirected Graph from G_in and G_out")

# Save the plot to the specified directory
# plt.savefig(r'C:\Users\labo\OneDrive - Nexus365\Diss\exploratory analsis\plots\G_hat.png', bbox_inches='tight', pad_inches=0.01)

# plt.title("Directed Graph from filtered_adjacency_normalised_out_df")
plt.show()

In [ ]:
# 2. Approach: Identifying initiators
# Filter records where position_in_wave is between 1 and 5
filtered_wave_liquidations = wave_liquidations_long[wave_liquidations_long['position_in_wave'].isin(range(1, 6))]

# Pivot the table to count positions for each address
position_counts = filtered_wave_liquidations.pivot_table(index='liquidation_caller_address', columns='position_in_wave', values='wave_ID', aggfunc='count').reset_index()

# Rename columns for clarity
position_counts.columns = ['liquidation_caller_address', 'position_1', 'position_2', 'position_3', 'position_4', 'position_5']

# Merge position counts with total liquidations
liquidator_stats = liquidator_stats.merge(position_counts, on='liquidation_caller_address', how='left').fillna(0)

# Order the dataframe by position 1 counts in descending order
liquidator_stats = liquidator_stats.sort_values(by='position_1', ascending=False).reset_index(drop=True)

# Calculate relative positions
liquidator_stats['position_1_relative'] = liquidator_stats['position_1'] / liquidator_stats['total_wave_liquidations']
liquidator_stats['position_1to3_relative'] = (liquidator_stats['position_1'] + liquidator_stats['position_2'] + liquidator_stats['position_3']) / liquidator_stats['total_wave_liquidations']
liquidator_stats['position_1to5_relative'] = (liquidator_stats['position_1'] + liquidator_stats['position_2'] + liquidator_stats['position_3'] + liquidator_stats['position_4'] + liquidator_stats['position_5']) / liquidator_stats['total_wave_liquidations']

# Compute additional columns for "liquidator_stats"

# Calculate 'total_isolated_liquidations'
liquidator_stats['total_isolated_liquidations'] = liquidator_stats['total_liquidations'] - liquidator_stats['total_wave_liquidations']

# Calculate 'min_position' and 'max_position'
liquidator_stats_min_positions = wave_liquidations_long.groupby('liquidation_caller_address')['position_in_wave'].min()
liquidator_stats_max_positions = wave_liquidations_long.groupby('liquidation_caller_address')['position_in_wave'].max()
liquidator_stats = liquidator_stats.merge(liquidator_stats_min_positions.rename('min_position'), left_on='liquidation_caller_address', right_index=True, how='left')
liquidator_stats = liquidator_stats.merge(liquidator_stats_max_positions.rename('max_position'), left_on='liquidation_caller_address', right_index=True, how='left')

# Calculate 'total_waves'
liquidator_stats_total_waves = wave_liquidations_long.groupby('liquidation_caller_address')['wave_ID'].nunique()
liquidator_stats = liquidator_stats.merge(liquidator_stats_total_waves.rename('total_waves'), left_on='liquidation_caller_address', right_index=True, how='left')

# Calculate 'own_follower'
wave_liquidations_long['next_address'] = wave_liquidations_long.groupby('wave_ID')['liquidation_caller_address'].shift(-1)
liquidator_stats_own_followers = wave_liquidations_long[wave_liquidations_long['liquidation_caller_address'] == wave_liquidations_long['next_address']].groupby('liquidation_caller_address').size()
liquidator_stats = liquidator_stats.merge(liquidator_stats_own_followers.rename('own_follower'), left_on='liquidation_caller_address', right_index=True, how='left')
liquidator_stats['own_follower'].fillna(0, inplace=True)

# Define a function to find the longest run of consecutive addresses
def longest_run_of_address(group):
    addresses = group['liquidation_caller_address'].tolist()
    
    # Identify the longest run of consecutive addresses
    liquidator_stats_longest_run = max((sum(1 for _ in g) for k, g in itertools.groupby(addresses)), default=0)
    return liquidator_stats_longest_run

# Compute the longest run for each address across all waves
liquidator_stats_longest_runs = wave_liquidations_long.groupby(['wave_ID', 'liquidation_caller_address']).apply(longest_run_of_address).groupby('liquidation_caller_address').max()
liquidator_stats = liquidator_stats.merge(liquidator_stats_longest_runs.rename('longest_run'), left_on='liquidation_caller_address', right_index=True, how='left')

# Calculate additional columns for "liquidator_stats"

# Calculate 'avg_liquidations_per_wave'
liquidator_stats['avg_liquidations_per_wave'] = liquidator_stats['total_wave_liquidations'] / liquidator_stats['total_waves']

# Define a function to compute 'total_runs'
def total_runs_of_address(group):
    addresses = group['liquidation_caller_address'].tolist()
    
    # Identify the runs of consecutive addresses
    runs = [(k, sum(1 for _ in g)) for k, g in itertools.groupby(addresses)]
    
    # Filter for runs of length 2 or more and count them
    return [k for k, length in runs if length >= 2]

# Compute the total runs for each address across all waves
liquidator_stats_total_runs = wave_liquidations_long.groupby('wave_ID').apply(total_runs_of_address)
liquidator_stats_total_runs_list = [item for sublist in liquidator_stats_total_runs for item in sublist]

# Count occurrences for each address
total_runs = pd.Series(liquidator_stats_total_runs_list).value_counts()

# Merge with liquidator_stats
liquidator_stats = liquidator_stats.merge(total_runs.rename('total_runs'), left_on='liquidation_caller_address', right_index=True, how='left')
liquidator_stats['total_runs'].fillna(0, inplace=True)

# Calculate 'total_run_liquidations'
liquidator_stats['total_run_liquidations'] = liquidator_stats['total_runs'] + liquidator_stats['own_follower']

# Define a function to get top 3 addresses and the rest for a given position column
def get_top_5_and_rest(df, column_name):
    """
    Extract top 5 addresses based on a column and group the rest.
    """
    top_5 = df.nlargest(5, column_name)
    rest_value = df[column_name].sum() - top_5[column_name].sum()
    values = top_5[column_name].tolist() + [rest_value]
    labels = top_5['liquidation_caller_address'].astype(str).tolist() + ['Others']
    percentages = [f"{(val/df[column_name].sum())*100:.1f}%" for val in values]
    return values, labels, percentages

# Define a plotting function
def plot_pie_chart(ax, values, labels, percentages, title):
    ax.pie(values, labels=[f"{label} ({percentage})" for label, percentage in zip(labels, percentages)], 
           startangle=90, colors=['steelblue', 'powderblue', 'skyblue', 'gray', 'darkgray', 'lightgray'],
           labeldistance=1.1, pctdistance=0.8)
    ax.set_title(title)

# Extract data for each position
values_1, labels_1, percentages_1 = get_top_5_and_rest(liquidator_stats, 'position_1')
values_2, labels_2, percentages_2 = get_top_5_and_rest(liquidator_stats, 'position_2')
values_3, labels_3, percentages_3 = get_top_5_and_rest(liquidator_stats, 'position_3')
values_4, labels_4, percentages_4 = get_top_5_and_rest(liquidator_stats, 'position_4')
values_5, labels_5, percentages_5 = get_top_5_and_rest(liquidator_stats, 'position_5')

# Plot the pie charts
fig, axs = plt.subplots(1, 2, figsize=(27, 10))

# Adjust the horizontal space between subplots
plt.subplots_adjust(wspace=2)

plot_pie_chart(axs[0], values_1, labels_1, percentages_1, 'Position 1')
plot_pie_chart(axs[1], values_2, labels_2, percentages_2, 'Position 2')
#plot_pie_chart(axs[2], values_3, labels_3, percentages_3, 'Position 3')
#plot_pie_chart(axs[3], values_4, labels_4, percentages_4, 'Position 4')
#plot_pie_chart(axs[4], values_5, labels_5, percentages_5, 'Position 5')

plt.tight_layout()
plt.show()

In [ ]:
# How many different starting tuples are there?

# Extract the first two nodes for each wave
first_two_nodes = wave_liquidations_long[wave_liquidations_long['position_in_wave'].isin([1, 2])]

# Group by wave_ID and aggregate the liquidation_caller_address into a tuple
starting_tuples = first_two_nodes.groupby('wave_ID')['liquidation_caller_address'].apply(tuple)

# Total number of UNIQUE starting tuples
num_unique_starting_tuples = len(set(starting_tuples))
print(f"Number of different starting tuples: {num_unique_starting_tuples}")

# How many double tuples (of the form (X,X)) exist?

# Count tuples with the same number for both positions
double_tuple_count = sum(1 for t in set(starting_tuples) if t[0] == t[1])
print(f"Number of double tuples: {double_tuple_count}")

# How many double tuples start a liquidation wave?

# Count the total occurrences of the repeated tuples in the original tuples series
double_tuple_occurrences = starting_tuples[starting_tuples.apply(lambda x: x[0] == x[1])].count()
print(f"Double tuples occur at {double_tuple_occurrences} or {(double_tuple_occurrences / total_waves):.2%} of liquidation wave starts.")

# Filter the tuples to get only the repeated tuples, get the counts and the top 10 addresses
top_10_starting_tuples = starting_tuples[starting_tuples.apply(lambda x: x[0] == x[1])].value_counts().head(10)

# Calculate the cumulative share
cumulative_share = top_10_starting_tuples.cumsum() / double_tuple_occurrences

# Convert the index elements to strings using a list comprehension
x_labels = [str(item) for item in top_10_starting_tuples.index]

# Create a figure with two y-axes
fig, ax1 = plt.subplots(figsize=(21, 14))
ax1.set_axisbelow(True)

# Create the bar chart on the first y-axis
ax1.bar(x_labels, top_10_starting_tuples.values, color='skyblue')
ax1.set_xlabel('Liquidator', labelpad=10)
ax1.set_ylabel('Number of Occurrences', labelpad=10)
ax1.tick_params(axis='y')
ax1.set_xticklabels([label.split(',')[0].strip('()') for label in x_labels], rotation=45)
ax1.tick_params(axis='x', which='major', length=20, width=4)

# Add a grid on the first y-axis
ax1.grid(axis='y', linestyle=':', linewidth=3)
ax1.tick_params(axis='y', which='major', length=20, width=4)

# Create the line plot on the second y-axis
ax2 = ax1.twinx()
ax2.plot(x_labels, cumulative_share, marker='o', markersize=16, linewidth=5, color='grey', linestyle='-', label='Cumulative Share')
ax2.set_ylabel('Cumulative Share')
ax2.tick_params(axis='y')
ax2.set_ylim(0, 1)  # Ensure the y-axis for cumulative share ranges from 0 to 1
ax2.tick_params(axis='y', which='major', length=20, width=4)

# Format the y-tick labels of the second axis as percentages
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=0))

# Add a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()

# Save the plot to the specified directory
plt.show()


In [ ]:
# Create a directed graph G_1: Universal weight of +1 for the first edge only after wave start (excluding self-edges)
G_1 = nx.DiGraph()

# Iterate over the wave_liquidations_long dataframe grouped by wave_ID
for _, group in wave_liquidations_long.sort_values(by='position_in_wave').groupby('wave_ID'):
    # Get the first two nodes of each wave
    nodes = group['liquidation_caller_address'].tolist()[:2]
    source, target = nodes

    # Avoid self-loops
    if source != target:
        # Check if the edge already exists
        if G_1.has_edge(source, target):
            # Increase the weight by +1
            G_1[source][target]['weight'] += 1
        else:
            # Add a new edge with weight = 1
            G_1.add_edge(source, target, weight=1)

# Number of nodes and edges in the updated G_1
num_nodes_G1 = G_1.number_of_nodes()
num_edges_G1 = G_1.number_of_edges()
print(f"Number of nodes in G_1: {num_nodes_G1}")
print(f"Number of edges in G_1: {num_edges_G1}")

# Plotting the directed graph G_1
plt.figure(figsize=(15, 15))
pos_1 = nx.spring_layout(G_1, k=1.5)
# pos_1 = nx.random_layout(G_1)
nx.draw_networkx_nodes(G_1, pos_1, node_size=200)
nx.draw_networkx_edges(G_1, pos_1, width=1.0, alpha=0.5, arrowstyle='-|>', arrowsize=20, edge_color='grey')
nx.draw_networkx_labels(G_1, pos_1, font_size=8)
plt.title("Directed Graph G_1 from wave_liquidations_long")
plt.axis('off')
plt.show()


In [ ]:
# Extract edges with weight > 1
edges_G_1_subgraph = [(u, v) for u, v, d in G_1.edges(data=True) if d["weight"] >= 15]

# Create a subgraph from G_1 using the desired edges
G_1_subgraph = G_1.edge_subgraph(edges_G_1_subgraph)

# Plotting the subgraph with uniform edge widths
plt.figure(figsize=(10, 10))
pos_G_1_subgraph = nx.circular_layout(G_1_subgraph)
nx.draw_networkx(G_1_subgraph, pos=pos_G_1_subgraph, with_labels=True, node_size=500, font_size=10, edge_color="black", width=1, connectionstyle="arc3, rad=0.08")
plt.title("Subgraph of G_1 with Edges of Weight > 15")  # Added a title for clarity
plt.axis('off')
plt.show()

# Create color map based on total_wave_liquidations values
color_map_G_1_subgraph = []

for node in G_1_subgraph.nodes:
    liquidations = liquidator_stats.loc[liquidator_stats['liquidation_caller_address'] == node, 'total_wave_liquidations']
    if len(liquidations) > 0:
        liquidations = liquidations.iloc[0]
        if liquidations < 250:
            color_map_G_1_subgraph.append('lightgrey')
        elif liquidations < 500:
            color_map_G_1_subgraph.append('skyblue')
        else:
            color_map_G_1_subgraph.append((0.27, 0.51, 0.71, 0.9))  # Steelblue with alpha 90%
    else:
        color_map_G_1_subgraph.append('gray')  # Default color for nodes not in liquidator_stats

# Plotting the subgraph with modified edge styles and two separate edge collections
plt.figure(figsize=(27, 9))
pos_G_1_subgraph = nx.spring_layout(G_1_subgraph, seed=3, k=2.4)

# Draw nodes and labels
nx.draw_networkx_nodes(G_1_subgraph, pos=pos_G_1_subgraph, node_size=4500, node_color=color_map_G_1_subgraph)
nx.draw_networkx_labels(G_1_subgraph, pos=pos_G_1_subgraph, font_size=30, font_family='serif')

# Separate edges by weight criteria and draw them with different styles
solid_edges_G_1_subgraph = [(u, v) for u, v, d in G_1_subgraph.edges(data=True) if d["weight"] >= 35]
dashed_edges_G_1_subgraph = [(u, v) for u, v, d in G_1_subgraph.edges(data=True) if d["weight"] < 35]

nx.draw_networkx_edges(G_1_subgraph, pos=pos_G_1_subgraph, edgelist=solid_edges_G_1_subgraph,
                       style="solid", connectionstyle="arc3,rad=0.1", width=4, arrowsize=30,
                       edge_color='black', alpha=0.5, min_target_margin=30, min_source_margin=32)
nx.draw_networkx_edges(G_1_subgraph, pos=pos_G_1_subgraph, edgelist=dashed_edges_G_1_subgraph,
                       style="dashed", connectionstyle="arc3,rad=0.1", width=4, arrowsize=30,
                       edge_color='black', alpha=0.5, min_target_margin=30, min_source_margin=32)

# Adding a legend
legend_items_G_1_subgraph = [
    Line2D([0], [0], color='black', alpha=0.5, linestyle='dashed', label='$\mathregular{Weight \in [15,35)}$'),
    Line2D([0], [0], color='black', alpha=0.5, linestyle='-', label='$\mathregular{Weight \geq 35}$'),
    Line2D([0], [0], marker='o', color='skyblue', label='$\mathregular{Wave\ liquidations \in [250,500)}$', markersize=10, linestyle='None'),
    Line2D([0], [0], marker='o', color='steelblue', label='$\mathregular{Wave\ liquidations \geq 500}$', markersize=10, linestyle='None')]

plt.legend(handles=legend_items_G_1_subgraph, loc='upper right', frameon=True, facecolor='white', framealpha=0.5).get_frame().set_linewidth(4.0)
plt.title("Subgraph of G_1 with Edges of Weight > 15")  # Updated the title
plt.axis('off')

# Save the plot to the specified directory
#plt.savefig(r'C:\Users\labo\OneDrive - Nexus365\Diss\exploratory analsis\plots\G_1_subgraph.png', bbox_inches='tight', pad_inches=0.01)

plt.show()
